In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset 

import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
#import zipfile
#import io
#zf = zipfile.ZipFile(io.BytesIO(uploaded['data.zip']), "r")
#zf.extractall()

#print(zf.infolist())

In [4]:
x_train = np.load('input/X_train_clean_140_45.npy', encoding='bytes')
y_train = np.loadtxt('input/train_labels.csv', delimiter=',', skiprows=1, dtype='str')

#x_test = np.load('X_test.npy', encoding='bytes')
middle = x_train.shape[0] // 2
x_train_, x_test_ = x_train[:middle], x_train[middle:]
y_train, y_test = y_train[:middle, 1], y_train[middle:, 1]
#middle = x_train.shape[0] // 2
#x_train_, x_test_ = x_train[:500], x_train[500:1000]
#y_train, y_test = y_train[:500, 1], y_train[500:1000, 1]

#ksize = 20
#chans = (100 // ksize) ** 2

#x_train = []
#for exi in range(x_train_.shape[0]):
#    ex = x_train_[exi].reshape((100,100))
#    x_train.append([])
#    for y in range(100 // ksize):
#        for x in range(100 // ksize):
#            x_train[exi].append(ex[x*ksize:(x+1)*ksize,y*ksize:(y+1)*ksize])

#x_test = []
#for exi in range(x_test_.shape[0]):
#    ex = x_test_[exi].reshape((100,100))
#    x_test.append([])
#    for y in range(100 // ksize):
#        for x in range(100 // ksize):
#            x_test[exi].append(ex[x*ksize:(x+1)*ksize,y*ksize:(y+1)*ksize])

x_train = np.array(x_train_.reshape((-1, 1, 100,100)))
x_test  = np.array(x_test_.reshape((-1, 1, 100,100)))


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5000, 1, 100, 100)
(5000,)
(5000, 1, 100, 100)
(5000,)


Le code suivant est adapté du tp d'introduction à PyTorch de IFT 3395

In [5]:
#iris = datasets.load_iris()

# separer dans train/test avec sklearn
#(x_train, x_test, y_train, y_test) = train_test_split(iris.data, iris.target)

num_train = x_train.shape[0]
num_test = x_test.shape[0]
num_features = x_train.shape[1]
num_classes = 31

# encodage one-hot
# e.g. 1 -> [0, 1, 0], 2 -> [0, 0 ,1]  
one_hot = OneHotEncoder(sparse=False)
y_train = one_hot.fit_transform(y_train.reshape(-1,1))
y_test = one_hot.transform(y_test.reshape(-1,1))

In [6]:
# I have an amd gpu :(
#device = torch.device('cuda:0')    # we use 0 to specify the first GPU

In [7]:
x_train = torch.from_numpy(x_train).float() #.to(device)
x_test  = torch.from_numpy(x_test).float() #.to(device)

y_train = np.argmax(y_train, axis=1)
y_test  = np.argmax(y_test, axis=1)

y_train = torch.from_numpy(y_train).long() #.to(device)
y_test  = torch.from_numpy(y_test).long() #.to(device)

In [8]:
torch.manual_seed(0)
#torch.cuda.manual_seed(0)

class HeavyNet(nn.Module):
    """A medium sized network that performs very well on MNIST."""
    
    def __init__(self):
        super().__init__()
        # conv block 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        # conv block 2
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        # fully connected layers
        self.fc1 = nn.Linear(64*25*25, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 31)
        
    def forward(self, x):
        # x is [batch_size, channels, heigth, width] = [bs, 1, 28, 28]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 32, 48, 48]
        #print(x.shape)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 64, 25, 25]
        #print(x.shape)
        x = x.view(x.size(0), -1) # flatten
        #print(x.shape)
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = HeavyNet()

# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')

num_epochs = 100
lr = 0.0005
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

def load_model(path):
    model.load_state_dict(torch.load(path))

def train():
    """Perform one epoch of training."""
    model.train()
    
    for epoch in range(num_epochs):
        
        idxs = np.random.uniform(0, x_train.shape[0], 50)
        x = x_train[idxs]
        y = y_train[idxs]
        
        # Let them code what's here
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        ###
        
        if ((epoch + 1) % 5) == 0:
            print('Train Epoch: {}\tloss: {:.6f}'.format((epoch+1), loss.item()))

_ = """
def train_model():
    num_epochs = 50
    lr = 0.0005
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    losses = []
    
    for epoch in range(num_epochs):
        idxs = np.random.uniform(0, x_train.shape[0], 25)
        
        x = x_train[idxs]
        y = y_train[idxs]
        
        preds = model(x)
        loss = loss_fn(preds, y)
        
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 5 == 0:
            print('Train Epoch {}: Loss: {:.6f}'.format(epoch, loss.item()))
    
    plot(range(num_epochs), losses)
    xlabel('epochs')
    ylabel('mean squared loss')
"""


In [9]:
def save_model():
    torch.save(model.state_dict(), 'model-conv2d-x_train-500')

In [10]:
#load_model('model-x_train-5000')
#train_model()

#preds = model(x_test)
#loss = loss_fn(preds, y_test)

#print('mean squared loss on test data %.7f %%' % (loss * 100.0))

#
# 5000 train, 5000 test, 500 epoch, 2e-5 learning rate
#     mean squared loss on test data 34.0440140 %
#

In [14]:
def test():
    """Evaluate the model by doing one pass over a dataset"""
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    with torch.no_grad():
        m = x_test.shape[0] // 100
        for k in range(m):
            x = x_test[k*m:(k+1)*m]
            y = y_test[k*m:(k+1)*m]
            
            # let them code what's here
            output = model(x)
            test_size += len(x)
            test_loss += loss_fn(output, y).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [12]:
train()

Train Epoch: 5	loss: 3.503602
Train Epoch: 10	loss: 3.385924
Train Epoch: 15	loss: 3.380804
Train Epoch: 20	loss: 3.409863
Train Epoch: 25	loss: 3.458301
Train Epoch: 30	loss: 3.281842
Train Epoch: 35	loss: 3.417306
Train Epoch: 40	loss: 3.356906
Train Epoch: 45	loss: 3.354928
Train Epoch: 50	loss: 3.285452
Train Epoch: 55	loss: 3.272003
Train Epoch: 60	loss: 3.274923
Train Epoch: 65	loss: 3.356913
Train Epoch: 70	loss: 3.239545
Train Epoch: 75	loss: 3.271949
Train Epoch: 80	loss: 3.206142
Train Epoch: 85	loss: 3.161509
Train Epoch: 90	loss: 3.233590
Train Epoch: 95	loss: 3.170251
Train Epoch: 100	loss: 3.148276


In [15]:
test()


Test set: Average loss: 0.0673, Accuracy: 203/2500 (8%)



(0.06726066446304321, 0.0812)